In [1]:
import Pkg
Pkg.activate("../../");

  Activating project at `C:\SY\MIT\UROP\InverseGraphics\InverseGraphics`


In [2]:
import Revise
import GLRenderer as GL
import Images as I
import MiniGSG as S
import Rotations as R
import PoseComposition: Pose, IDENTITY_POSE, IDENTITY_ORN
import InverseGraphics as T
import NearestNeighbors
import LightGraphs as LG
import StaticArrays
import ProgressMeter
using Distributions
import Gen
using Plots
using ProgressMeter
try
    import MeshCatViz as V
catch
    import MeshCatViz as V    
end

In [3]:
# # Initialize the renderer
# V.setup_visualizer()

In [4]:
# Loading the YCB object models
YCB_DIR = joinpath(dirname(dirname(pwd())),"data")
world_scaling_factor = 10.0
id_to_cloud, id_to_shift, id_to_box  = T.load_ycbv_models_adjusted(YCB_DIR, world_scaling_factor);
all_ids = sort(collect(keys(id_to_cloud)));
names = T.load_ycb_model_list(YCB_DIR)

21-element Vector{String}:
 "002_master_chef_can"
 "003_cracker_box"
 "004_sugar_box"
 "005_tomato_soup_can"
 "006_mustard_bottle"
 "007_tuna_fish_can"
 "008_pudding_box"
 "009_gelatin_box"
 "010_potted_meat_can"
 "011_banana"
 "019_pitcher_base"
 "021_bleach_cleanser"
 "024_bowl"
 "025_mug"
 "035_power_drill"
 "036_wood_block"
 "037_scissors"
 "040_large_marker"
 "051_large_clamp"
 "052_extra_large_clamp"
 "061_foam_brick"

In [5]:
# Initialize the canera intrinsics and renderer that will render using those intrinsics.
camera = GL.CameraIntrinsics()
renderer = GL.setup_renderer(camera, GL.DepthMode())
resolution = 0.05
# for id in all_ids
#     cloud = id_to_cloud[id]
#     mesh = GL.mesh_from_voxelized_cloud(GL.voxelize(cloud, resolution), resolution);
#     GL.load_object!(renderer, mesh)
# end


box_mesh = GL.box_mesh_from_dims([2.0, 2.0, 2.0])
GL.load_object!(renderer, box_mesh)

@show camera;

gl_version_for_shaders = "410"
true
camera = GLRenderer.CameraIntrinsics
  width: Int64 640
  height: Int64 480
  fx: Float64 640.0
  fy: Float64 640.0
  cx: Float64 320.0
  cy: Float64 240.0
  near: Float64 0.001
  far: Float64 100.0



In [6]:
# Also to make nice visualizations, you can rendering RGB mode!

In [7]:
# renderer = GL.setup_renderer(camera, GL.RGBMode())
# obj_paths = T.load_ycb_model_obj_file_paths(YCB_DIR)
# for id in all_ids
#     mesh = GL.get_mesh_data_from_obj_file(obj_paths[id])
#     mesh = T.scale_and_shift_mesh(mesh, world_scaling_factor, id_to_shift[id])
#     GL.load_object!(renderer, mesh)
# end

## introductory visualizations
to get a better sense of resolution effect on likelihood + qualitative results 

In [8]:
# #########
# # radius scaling
# #########
# bounds = (-100.0, 100.0, -100.0, 100.0,-100.0,100.0)

# i = 1
# pose = Pose([0.0, 0.0, 3.0], R.RotXYZ(0.4, -0.2, 0.4))

# # Reset the intrinsics inside of the renderer.
# GL.set_intrinsics!(renderer, camera) 

# # And render the same image as above.
# gt_depth_image = GL.gl_render(renderer, [i], [pose], IDENTITY_POSE)
# IJulia.display(GL.view_depth_image(gt_depth_image))

# # Create point cloud corresponding to that rendered depth image.
# gt_cloud = GL.depth_image_to_point_cloud(gt_depth_image, camera)
# # print(size(gt_cloud))


# @show i

# radius = 0.01  # "tight" radius
# sampled_cloud = Gen.random(T.uniform_mixture_from_template, gt_cloud, 0.0001, radius, bounds)
# # Visualize that point cloud.
# V.setup_visualizer()
# V.viz(sampled_cloud)  # = V.viz(T.move_points_to_frame_b(c, camera_pose))

 
# radius = float(1)  # "loose" radius (can still pick out general shape)
# sampled_cloud = Gen.random(T.uniform_mixture_from_template, gt_cloud, 0.0001, radius, bounds)
# # Visualize that point cloud.
# V.setup_visualizer()
# V.viz(sampled_cloud)  # = V.viz(T.move_points_to_frame_b(c, camera_pose))

 
# radius = float(10)  # very loose radius; points on a sphere (w/ some concentration on center with shape of obj)
# sampled_cloud = Gen.random(T.uniform_mixture_from_template, gt_cloud, 0.0001, radius, bounds)
# # # Visualize that point cloud.
# # V.setup_visualizer()
# # V.viz(sampled_cloud)  # = V.viz(T.move_points_to_frame_b(c, camera_pose))

In [9]:
# ########
# # camera downscaling
# ########
# using IJulia

# final = 24
# for downscale_factor=1:3.0:final
#     println("scale down by $downscale_factor")
    
#     # scale camera according to resolution; reset renderer
#     scaled_camera = GL.scale_down_camera(camera, downscale_factor)
        
#     # Set the renderer to now have those scaled down intrinsics.
#     GL.set_intrinsics!(renderer, scaled_camera)
 
#     d = GL.gl_render(renderer, [i], [pose], IDENTITY_POSE)

#     img = GL.view_depth_image(d)
#     img = I.imresize(img, (camera.height, camera.width));
#     IJulia.display(img)
  
#     # revert intrinsics
#     GL.set_intrinsics!(renderer, camera)

# end


# Intro demo of coarse-to-fine with particle filtering

## Particle Filtering + Orientation enumeration

### Orientation enumeration

In [10]:
function fibonacci_sphere(samples)
    points = []
    phi = π * (3. - sqrt(5.))
 
    for i in 0:(samples-1)
        y = 1 - (i / Float64(samples - 1)) * 2
        #@show y
        radius = sqrt(1 - y * y)

        theta = phi * i

        x = cos(theta) * radius
        z = sin(theta) * radius

        push!(points, (x, y, z))
    end
    return points
end

fibonacci_sphere (generic function with 1 method)

In [42]:
unit_sphere_directions = fibonacci_sphere(25);
other_rotation_angle = collect(0:0.2:(2*π));

rotations_to_enumerate_over = [
    let
        T.geodesicHopf_select_axis(StaticArrays.SVector(dir), ang, 1)
    end
    for dir in unit_sphere_directions, 
        ang in other_rotation_angle
];
num_rotations_to_enumerate_over = length(rotations_to_enumerate_over);

### Helpers

In [43]:
function viz_trace(trace)
    V.setup_visualizer()
#     V.reset_visualizer()
    V.viz(Gen.get_retval(trace).voxelized_cloud  ./ 10.0; color=I.colorant"red", channel_name=:gen);
    V.viz(Gen.get_retval(trace).obs_cloud ./ 10.0; color=I.colorant"blue", channel_name=:obs);
end

viz_trace (generic function with 1 method)

In [77]:
"""adapted from gen pf_step:
Perform a particle filter update, where the model arguments are adjusted, new observations are added, and the default proposal is used for new latent state.
"""
function particle_filter_step!(state::Gen.ParticleFilterState{U}, new_args::Tuple, argdiffs::Tuple,
        observations) where {U}    
    
    log_incremental_weights = Vector{Float64}(undef, num_particles) 
    radius::Float64 = new_args[1]
    @showprogress for i=1:num_particles
        # TODO? do mh, drift moves, etc. tune particle before update (i.e. new likelihood)
        #
        #
        
        # construct observations 
        particle_ori = Gen.get_retval(state.traces[i]).ori
#         observations = observations_func(particle_ori)
         
        
        # evolve the particle (with new radius involved in new_args)
        (state.new_traces[i], increment, _, discard) = Gen.update(
            state.traces[i], new_args, argdiffs, observations)
#         if !isempty(discard)
#             error("Choices were updated or deleted inside particle filter step: $discard")
#         end
        log_incremental_weights[i] = increment
        state.log_weights[i] += increment
    end
    
    # swap references
    tmp = state.traces
    state.traces = state.new_traces
    state.new_traces = tmp

    return log_incremental_weights
end

particle_filter_step!

In [78]:
bounds = (-100.0, 100.0, -100.0, 100.0,-100.0,300.0)
v_resolution = 0.2  # distance between two points on the pointcloud 

"""sample point clouds at the given camera resolution and radius"""
Gen.@gen function model(radius, renderer)
    num_candidate_objs = 6
    
    # fixed object (TODO: generalize to any obj id) and pose
    i = {:id} ~ Gen.categorical(fill(1.0/num_candidate_objs, (num_candidate_objs,)))
    p = {T.floating_pose_addr(1)} ~ T.uniformPose(-0.001, 0.001, -0.001, 0.001, 4.999, 5.001)
    
    
    rendered_depth = GL.gl_render(renderer, [i], [p], IDENTITY_POSE)
    gt_cloud = GL.depth_image_to_point_cloud(rendered_depth, renderer.camera_intrinsics)
    voxelized_cloud = GL.voxelize(gt_cloud, v_resolution)
    obs_cloud = {:obs => :cloud} ~ T.uniform_mixture_from_template(voxelized_cloud, 0.0001, radius, bounds)
#     print("."); flush(stdout)
         
    (id=i, pose=p, ori=p.orientation, cloud=gt_cloud, voxelized_cloud=voxelized_cloud, obs_cloud=obs_cloud)

end
 
"""return scale down factor of camera given desired resolution"""
function scale_factor(resolution, final_resolution)
   return final_resolution-resolution+1 
end

scale_factor

In [91]:
"""Selecting from a set of traces over all enumerated angles, generate a set of initial particles"""
Gen.@gen function generate_initial_pf_state(scored_traces, num_particles, U=Gen.DynamicDSLTrace{Gen.DynamicDSLFunction{Any}})
    # unpack traces
    traces = (t -> t[1]).(scored_traces)
    log_weights = (t -> t[2]).(scored_traces) #; log_weights = log_weights .- Gen.logsumexp(log_weights)  # ∝ likelihood
    
    _, norm_log_weights = Gen.normalize_weights(log_weights)       
    
    # sample initial set of particles
    selected_traces = Vector{Gen.DynamicDSLTrace{Gen.DynamicDSLFunction{Any}}}(undef, num_particles)
    selected_log_weights = Vector{Float64}(undef, num_particles)
    for i=1:num_particles
        trace_idx = {:t_idx => i} ~ Gen.categorical(exp.(norm_log_weights))   
        selected_traces[i] = traces[trace_idx]
        selected_log_weights[i] = log_weights[trace_idx]
    end
    
    println("Initial particle state computed"); flush(stdout)
    sleep(5)

    # visualize pdf of initial weights
    plot(selected_log_weights, seriestype=:stephist, fmt = :png)
    
    # intermediate results
    _, norm_log_weights = Gen.normalize_weights(selected_log_weights)
    best_trace = traces[argmax(norm_log_weights)];
    viz_trace(best_trace);
    
    sleep(5)
    println(sort(norm_log_weights[:])[end-5:end]); 
    
    
    
    flush(stdout)
    
    return Gen.ParticleFilterState{U}(selected_traces, Vector{U}(undef, num_particles), selected_log_weights, 0., collect(1:num_particles))

end

generate_initial_pf_state

### control resolution with radius (fixed camera intrinsics)

In [92]:
"""Particle filter on the evolution of agent knowledge over time; resolution control with radius"""


function particle_filter(renderer, init_radius::Float64, final_radius::Float64, enum_ori_traces,
                        gt_obj_id::Int, gt_pose::Pose, num_particles::Int, num_samples::Int, U=Gen.DynamicDSLTrace{Gen.DynamicDSLFunction{Any}})
    
    # initialize renderer and particle filter
    GL.set_intrinsics!(renderer, camera) 
    gt_depth_image = GL.gl_render(renderer, [gt_object_id], [gt_pose], IDENTITY_POSE)
    gt_cloud = GL.depth_image_to_point_cloud(gt_depth_image, renderer.camera_intrinsics)
    gt_voxelized_cloud = GL.voxelize(gt_cloud, v_resolution)
    V.setup_visualizer()
    V.viz(gt_voxelized_cloud)
    println("\ninitializing particle filter at radius $init_radius") 
    flush(stdout)
    
    
    # initialize a state of initial particles with various rotations
    scored_traces = enum_ori_traces  # get precomputed
    
    state::Gen.ParticleFilterState{U} = generate_initial_pf_state(scored_traces, num_particles)
    
    
    # evolve over resolutions (modify sphere radius `r` of mixture point cloud likelihood)
    @assert(final_radius <= init_radius)
    observations = Gen.choicemap((:obs => :cloud) => gt_voxelized_cloud, :id => gt_obj_id)
    step = -0.05 
    for radius in init_radius+step:step:final_radius
        println("\n========Radius=$radius========"); flush(stdout)
        Gen.maybe_resample!(state, ess_threshold=num_particles/3, verbose=true)  
        
        # update pf 
        current_log_weights = particle_filter_step!(state, (radius, renderer), (Gen.UnknownChange(),), observations)    
        
        # see intermediate results
        top_n = 10
        _, norm_log_weights = Gen.normalize_weights(current_log_weights)
        p = sortperm(norm_log_weights)[end-top_n:end]; 
        
        best_trace = state.traces[argmax(norm_log_weights)];
        viz_trace(best_trace);
        println("top$top_n traces:")
        for idx in p
           println("current weight=", norm_log_weights[idx], "\t orientation=", Gen.get_retval(state.traces[idx]).ori[1:3]) 
        end
        flush(stdout)
        
    end;
    
     return Gen.sample_unweighted_traces(state, num_samples)
    
end

particle_filter (generic function with 4 methods)

In [93]:
# Initialize the canera intrinsics and renderer that will render using those intrinsics.
GL.set_intrinsics!(renderer, camera)
 
init_radius, final_radius = float(1), float(0.1)  # low to high "focus"

gt_object_id = 1   # cube
gt_pose = T.Pose(pos=[0.0, 0.0, 5.0], orientation=QuatRotation(0.2, -0.5, 0.1, -0.8))


function precompute_enumerations(gt_cloud, gt_pose, init_radius, renderer, rotations_to_enumerate_over)
    # initialize a state of initial particles with various rotations
    println("Enumerating over all angles:")
    scored_traces = @showprogress map(orn -> 
                    Gen.generate(model, (init_radius, renderer),                 
                                Gen.choicemap((:obs => :cloud) => gt_cloud, 
                                            :id => 1, 
                                            T.floating_pose_addr(1) => Pose(gt_pose.pos, orn))),
                                rotations_to_enumerate_over[:]);   
    return scored_traces
end 
enum_ori_traces = precompute_enumerations(gt_voxelized_cloud, gt_pose, init_radius, renderer, rotations_to_enumerate_over)

num_rotations_to_enumerate_over = length(enum_ori_traces)
@show num_rotations_to_enumerate_over
num_particles = 500
num_samples = num_particles
@time pf_traces = particle_filter(renderer, init_radius, final_radius, enum_ori_traces, gt_object_id, gt_pose, num_particles, num_samples);


true
num_rotations_to_enumerate_over = 800
true

initializing particle filter at radius 1.0
Initial particle state computed
[-5.780065703163018, -5.780065703163018, -5.780065703163018, -5.780065703163018, -5.780065703163018, -5.780065703163018]

========Radius=0.95========


┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8731
└ @ MeshCat C:\Users\karen\.julia\packages\MeshCat\Ax8pH\src\visualizer.jl:73
┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8732
└ @ MeshCat C:\Users\karen\.julia\packages\MeshCat\Ax8pH\src\visualizer.jl:73
Progress: 100%|█████████████████████████████████████████| Time: 0:00:21


effective sample size: 359.024604170306, doing resample: false
top10 traces:
current weight=-4.979509647520359	 orientation=[-0.8652112097532296, 2.7755575615628914e-17, -0.5014075812324267]
current weight=-4.979509647520359	 orientation=[-0.8652112097532296, 2.7755575615628914e-17, -0.5014075812324267]
current weight=-4.979509647520359	 orientation=[-0.8652112097532296, 2.7755575615628914e-17, -0.5014075812324267]
current weight=-4.979509647520359	 orientation=[-0.8652112097532296, 2.7755575615628914e-17, -0.5014075812324267]
current weight=-4.979509647520359	 orientation=[-0.8652112097532296, 2.7755575615628914e-17, -0.5014075812324267]
current weight=-4.979509647520359	 orientation=[-0.8652112097532296, 2.7755575615628914e-17, -0.5014075812324267]
current weight=-4.979509647520359	 orientation=[-0.8652112097532296, 2.7755575615628914e-17, -0.5014075812324267]
current weight=-4.612990854920241	 orientation=[0.08771630578430317, -0.9166666666666669, -0.3899077736103459]
current weight

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8733
└ @ MeshCat C:\Users\karen\.julia\packages\MeshCat\Ax8pH\src\visualizer.jl:73
Progress: 100%|█████████████████████████████████████████| Time: 0:00:21


effective sample size: 397.5102915700524, doing resample: false
top10 traces:
current weight=-5.770186296780452	 orientation=[0.08771630578430328, -0.916666666666667, -0.389907773610346]
current weight=-5.770186296780452	 orientation=[0.08771630578430328, -0.916666666666667, -0.389907773610346]
current weight=-5.770186296780452	 orientation=[0.08771630578430328, -0.916666666666667, -0.389907773610346]
current weight=-5.770186296780452	 orientation=[0.08771630578430328, -0.916666666666667, -0.389907773610346]
current weight=-5.770186296780452	 orientation=[0.08771630578430328, -0.916666666666667, -0.389907773610346]
current weight=-5.7489142377370115	 orientation=[0.08771630578430323, -0.9166666666666669, -0.3899077736103458]
current weight=-5.610176986761957	 orientation=[0.08771630578430317, -0.9166666666666669, -0.3899077736103459]
current weight=-5.610176986761957	 orientation=[0.08771630578430317, -0.9166666666666669, -0.3899077736103459]
current weight=-5.610176986761957	 orientat

...

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8740
└ @ MeshCat C:\Users\karen\.julia\packages\MeshCat\Ax8pH\src\visualizer.jl:73
Progress: 100%|█████████████████████████████████████████| Time: 0:00:21


effective sample size: 354.44644613573007, doing resample: false
top10 traces:
current weight=-5.778390823662008	 orientation=[0.08771630578430328, -0.916666666666667, -0.389907773610346]
current weight=-5.778390823662008	 orientation=[0.08771630578430328, -0.916666666666667, -0.389907773610346]
current weight=-5.778390823662008	 orientation=[0.08771630578430328, -0.916666666666667, -0.389907773610346]
current weight=-5.778390823662008	 orientation=[0.08771630578430328, -0.916666666666667, -0.389907773610346]
current weight=-5.778390823662008	 orientation=[0.08771630578430328, -0.916666666666667, -0.389907773610346]
current weight=-5.537134097103049	 orientation=[-0.2946914091498113, 0.9166666666666664, 0.2699614705759335]
current weight=-4.494620272301759	 orientation=[0.08771630578430331, -0.9166666666666669, -0.38990777361034584]
current weight=-4.091296861858851	 orientation=[0.08771630578430323, -0.9166666666666669, -0.3899077736103458]
current weight=-3.311401322930365	 orientati

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8741
└ @ MeshCat C:\Users\karen\.julia\packages\MeshCat\Ax8pH\src\visualizer.jl:73
Progress:  33%|██████████████                           |  ETA: 0:00:15

effective sample size: 333.4981339078575, doing resample: false


LoadError: InterruptException:

In [85]:
gt_pose

Pose⟨pos=[0.0, 0.0, 5.0], orientation=(w=0.20628424925175867, x=-0.5157106231293966, y=0.10314212462587934, z=-0.8251369970070347)⟩

In [66]:
QuatRotation(0.217001, -0.484508, 0.132558, -0.837013)

3×3 QuatRotation{Float64} with indices SOneTo(3)×SOneTo(3)(QuaternionF64(0.217001, -0.484508, 0.132558, -0.837013, true)):
 -0.436325   0.234815   0.86861
 -0.491716  -0.870678  -0.0116281
  0.753549  -0.432183   0.495361

In [67]:
new_pose = T.Pose(pos=[0.0, 0.0, 5.0],
                orientation=QuatRotation(0.2, -0.5, 0.1, -0.8))

Pose⟨pos=[-0.0003268263967760272, -2.2696846958917714e-5, 4.99912328284441], orientation=(w=0.2170010193567088, x=-0.48450804321860386, y=0.13255801182430774, z=-0.8370130746624066)⟩

In [38]:
PyCall.py"""
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
def run_viz(x,y,z,c):
    fig = plt.figure(figsize=(9, 6))
    ax = plt.axes(projection='3d')
    ax.scatter3D(x, y, z, c=c)
    p = ax.set_title("3D scatterplot", pad=25, size=15)
    ax.set_xlabel("X") 
    ax.set_ylabel("Y") 
    ax.set_zlabel("Z")
    plt.show()
"""

PyCall.py"run_viz"( (i->i[1]).(xyz), (i->i[2]).(xyz), (i->i[3]).(xyz), weights_xyz)

LoadError: LoadError: UndefVarError: PyCall not defined
in expression starting at In[38]:1

### control resolution with camera intrinsics (fixed radius)

In [19]:
# """Particle filter on the evolution of agent knowledge over time; resolution control with radius"""
# function particle_filter(camera, init_resolution, final_resolution, gt_obj_id, gt_pose, num_particles::Int, num_samples::Int)

#     # initialize particle filter
#     init_obs = Gen.choicemap()
#     GL.set_intrinsics!(renderer, camera) # initial renderer
#     init_gt_depth_image = GL.gl_render(renderer, [gt_object_id], [gt_pose], IDENTITY_POSE)
#     init_obs[:obs => init_resolution => :cloud] = GL.depth_image_to_point_cloud(init_gt_depth_image, camera)
#     GL.view_depth_image(init_gt_depth_image)
    
#     print("initializing particle filter at resolution $init_resolution")
#     state = Gen.initialize_particle_filter(cloud_sampler, 
#                             (init_resolution, renderer), init_obs, num_particles)

#     # evolve over resolutions (coarsen by scaling down the resolution of the rendered image)

#     for resolution in init_resolution+1:final_resolution
#         println("Camera Resolution $resolution")
#         Gen.maybe_resample!(state, ess_threshold=num_particles/3, verbose=true)
#         obs = Gen.choicemap() 
        
#         # scale camera according to resolution; reset renderer
#         scaled_camera = GL.scale_down_camera(camera, scale_factor(resolution, final_resolution))
        
#         # Set the renderer to now have those scaled down intrinsics.
#         GL.set_intrinsics!(renderer, scaled_camera)

#         # render depth image and revert intrinsics afterwards
#         gt_depth_image = GL.gl_render(renderer, [gt_object_id], [gt_pose], IDENTITY_POSE)
#         GL.set_intrinsics!(renderer, camera)
        
#         # Create GT (latent) point cloud corresponding to that rendered depth image.
#         obs[:obs => resolution => :cloud] = GL.depth_image_to_point_cloud(gt_depth_image, scaled_camera)
#         Gen.particle_filter_step!(state, (resolution, renderer, scaled_camera), (UnknownChange(),), obs)
        
# #         # Debug: Visualize gt point cloud.
# #         V.viz(c)  # = V.viz(T.move_points_to_frame_b(c, camera_pose))
# #         img = GL.view_depth_image(gt_depth_image)
# #         img = I.imresize(img, (camera.height, camera.width))
        
#         pf_traces = Gen.sample_unweighted_traces(state, num_samples)
        
# #         # Debug: current results
# #         for pf_trace in pf_traces
# #             # visualize intermediate result?
# #             # get_score(pf_trace)
# #         end
        
#     end;
    
#      return Gen.sample_unweighted_traces(state, num_samples)
# end;



# # Initialize the canera intrinsics and renderer that will render using those intrinsics.

# # try the drill example
# println("entering pf (camera scaling gradient)")
# gt_object_id = 15
# gt_pose = Pose([0.0, 0.0, 5.0], R.RotXYZ(0.1, 0.2, -0.4))

# init_resolution, final_resolution = float(1), float(10)  # low to high res
# num_particles = 10
# num_samples = 5
# @time pf_traces = particle_filter(camera, init_resolution, final_resolution, gt_object_id, gt_pose, num_particles, num_samples);
